In [118]:
import sys
import math
import os
import numpy as np
import pandas as pd
import tensorflow_addons
from matplotlib import pyplot as plt
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.backend import clear_session
from keras.callbacks import LearningRateScheduler
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import BatchNormalization, LayerNormalization
from tensorflow_addons.layers import InstanceNormalization, GroupNormalization
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [75]:
root_path = os.path.abspath(os.getcwd())

In [6]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX_, trainY_), (testX, testY) = cifar10.load_data()
	trainX = trainX_[:45000]
	trainY = trainY_[:45000]
	valX = trainX_[45000:]
	valY = trainY_[45000:]
	# one hot encode target values
	trainY = to_categorical(trainY)
	valY = to_categorical(valY)
	testY = to_categorical(testY)
	return trainX, trainY, valX, valY, testX, testY

In [7]:
# load train and test dataset
def load_dataset():
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    np.random.seed(0)
    val_idx = np.random.choice(range(trainX.shape[0]),5000,replace=False)
    valX = trainX[val_idx]
    valY = trainY[val_idx]

    trainX = np.delete(trainX,val_idx,0)
    trainY = np.delete(trainY,val_idx,0)

    trainY = to_categorical(trainY)
    valY = to_categorical(valY)
    testY = to_categorical(testY)
    return trainX, trainY, valX, valY, testX, testY

In [8]:
# scale pixels
def prep_pixels(train, val, test, standardize_data=False):
    # convert from integers to floats
    train_norm = train.astype('float32')
    val_norm = val.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    if standardize_data == True:
        mean = np.mean(trainX, 0)
        std = np.std(trainX, 0)
        train_stand = (train_norm - mean) / std
        val_stand = (val_norm - mean) / std
        test_stand = (test_norm - mean) / std
        return train_stand, val_stand, test_stand
    else:
        train_norm = train_norm / 255.0
        val_norm = val_norm / 255.0
        test_norm = test_norm / 255.0
        return train_norm, val_norm, test_norm

In [9]:
def loadData(reduce_data=False, standardize_data=False):
    trainX, trainY, valX, valY, testX, testY = load_dataset()
    trainX, valX, testX = prep_pixels(trainX, valX, testX, standardize_data)
    if reduce_data==True:
        return trainX[:10000], trainY[:10000], valX, valY, testX, testY
    else:
        return trainX, trainY, valX, valY, testX, testY

In [101]:
trainX, trainY, valX, valY, testX, testY = loadData(reduce_data=True)
print("trainX shape: ", trainX.shape)
print("trainY shape: ", trainY.shape)
print("valX shape: ", valX.shape)
print("valY shape: ", valY.shape)
print("testX shape: ", testX.shape)
print("testY shape: ", testY.shape)

trainX shape:  (10000, 32, 32, 3)
trainY shape:  (10000, 10)
valX shape:  (5000, 32, 32, 3)
valY shape:  (5000, 10)
testX shape:  (10000, 32, 32, 3)
testY shape:  (10000, 10)


In [11]:
def scheduler(n_epochs, lr):
    if n_epochs < 10:    
        return lr
    else:
        return lr * math.exp(-0.1)

In [139]:
def define_model(lr, optimizer, dropout_rate, lamda, normalization="batch"):

	clear_session()

	if optimizer == SGD:
		opt = SGD(lr, momentum = 0.9)
	elif optimizer == Adam:
		opt = Adam(lr)

	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(lamda), input_shape=(32, 32, 3)))
	if normalization == "batch":
		model.add(BatchNormalization())
	elif normalization == "layer":
		model.add(LayerNormalization())
	elif normalization == "group":
		model.add(GroupNormalization())
	elif normalization == "instance":
		model.add(InstanceNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(lamda)))
	if normalization == "batch":
		model.add(BatchNormalization())
	elif normalization == "layer":
		model.add(LayerNormalization())
	elif normalization == "group":
		model.add(GroupNormalization())
	elif normalization == "instance":
		model.add(InstanceNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(dropout_rate))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(lamda)))
	if normalization == "batch":
		model.add(BatchNormalization())
	elif normalization == "layer":
		model.add(LayerNormalization())
	elif normalization == "group":
		model.add(GroupNormalization())
	elif normalization == "instance":
		model.add(InstanceNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(lamda)))
	if normalization == "batch":
		model.add(BatchNormalization())
	elif normalization == "layer":
		model.add(LayerNormalization())
	elif normalization == "group":
		model.add(GroupNormalization())
	elif normalization == "instance":
		model.add(InstanceNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(dropout_rate))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(lamda)))
	if normalization == "batch":
		model.add(BatchNormalization())
	elif normalization == "layer":
		model.add(LayerNormalization())
	elif normalization == "group":
		model.add(GroupNormalization())
	elif normalization == "instance":
		model.add(InstanceNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(lamda)))
	if normalization == "batch":
		model.add(BatchNormalization())
	elif normalization == "layer":
		model.add(LayerNormalization())
	elif normalization == "group":
		model.add(GroupNormalization())
	elif normalization == "instance":
		model.add(InstanceNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(dropout_rate))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(lamda)))
	if normalization == "batch":
		model.add(BatchNormalization())
	elif normalization == "layer":
		model.add(LayerNormalization())
	elif normalization == "group":
		model.add(GroupNormalization())
	elif normalization == "instance":
		model.add(InstanceNormalization())
	model.add(Dropout(dropout_rate))
	model.add(Dense(10, activation='softmax'))
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

	return model

In [147]:
def summarize_diagnostics(history, var):

	plt.subplot(211)
	plt.title('Cross Entropy Loss')
	plt.plot(history.history['loss'], color='blue', label='train')
	plt.plot(history.history['val_loss'], color='orange', label='val')
	plt.subplot(212)
	plt.title(f'Classification Accuracy: {max(history.history["val_accuracy"])*100:.2f}%')
	plt.plot(history.history['accuracy'], color='blue', label='train')
	plt.plot(history.history['val_accuracy'], color='orange', label='val')
	plt.tight_layout()
	plt.savefig(os.path.join(root_path, 'Plots_jan', f'10 Epochs, variation={var}, Accuracy={max(history.history["val_accuracy"])*100:.2f}%.png'))
	#plt.show()
	plt.close()

In [143]:
def runNetwork(n_epochs, n_batch, lr=0.001, optimizer=SGD, dropout_rate=0, lamda=0, normalization=None, datagenerator=None, augmentation=False, standardize_data=False, reduce_data=False):
	trainX, trainY, valX, valY, testX, testY = loadData(reduce_data, standardize_data)

	if augmentation == True:
		model = define_model(lr, optimizer, dropout_rate, lamda, normalization)
		if datagenerator==None:
			datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, vertical_flip=True)
		else:
			datagen = datagenerator
		it_train = datagen.flow(trainX, trainY, batch_size=n_batch)
		steps = int(trainX.shape[0] / n_batch)
		callback = LearningRateScheduler(scheduler)
		history = model.fit(it_train, steps_per_epoch=steps, epochs=n_epochs, callbacks=[callback], validation_data=(valX, valY), verbose=1)
	else:
		model = define_model(lr, optimizer, dropout_rate, lamda, normalization)
		callback = LearningRateScheduler(scheduler)
		history = model.fit(trainX, trainY, epochs=n_epochs, batch_size=n_batch, callbacks=[callback], validation_data=(testX, testY), verbose=1)

	_, acc = model.evaluate(valX, valY, verbose=0)
	print('> %.3f' % (acc * 100.0))
	#summarize_diagnostics(history, normalization)

	return model, history

In [ ]:
model, history = runNetwork(n_epochs=1, n_batch=64, lr=.001, optimizer=Adam, dropout_rate=0.2, lamda=.005, normalization="instance", augmentation=True, standardize_data=True, reduce_data=True)

In [ ]:
norms = ["batch", "layer", "group", "instance"]
Historys_norm = []

for norm in norms:
    model, history = runNetwork(n_epochs=10, n_batch=64, lr=.001, optimizer=Adam, dropout_rate=0.2, lamda=.005, normalization=norm, augmentation=True, standardize_data=True, reduce_data=True)
    Historys_norm.append(history)
    summarize_diagnostics(history, norm)
    

In [ ]:
datagen1 = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, vertical_flip=True)
datagen2 = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, rotation_range=90)
datagen3 = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, zoom_range=1.5)
datagen4 = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, channel_shift_range=4.5)

datagens = [datagen1, datagen2, datagen3, datagen4]
Historys = []

for datagen in datagens:
    model, history = runNetwork(n_epochs=10, n_batch=64, lr=.001, optimizer=Adam, dropout_rate=0.2, lamda=.005, normalization="batch", datagenerator=datagen, augmentation=True, standardize_data=True, reduce_data=True)
    Historys.append(history)
    summarize_diagnostics(history)

In [ ]:
drops = [0.1, 0.2, 0.3, 0.4]
Historys_drop = []

for drop in drops:
    model, history = runNetwork(n_epochs=10, n_batch=64, lr=.001, optimizer=Adam, dropout_rate=drop, lamda=.005, normalization="batch", augmentation=True, standardize_data=True, reduce_data=True)
    Historys_drop.append(history)
    summarize_diagnostics(history, drop)

In [ ]:
%conda install tensorflow-gpu==2.0.0-alpha0